## Creating the Optimal Gov's Ball Scedule

In [2]:
#Bring in the neccessary libraries 
import pandas as pd
from io import StringIO
import urllib
from bs4 import BeautifulSoup 
import sqlite3 
import time
import numpy 
import signal
from urllib.request import Request, urlopen
from IPython.display import clear_output
import numpy as np

In [4]:
# Load package for progress bar
%run "progress_bar.py"
print ('Progress Bar Loaded')

Progress Bar Loaded


### Get Gov's Ball Artist Info

In [ ]:
#Grab the artist info from the Gov's ball website
with urllib.request.urlopen("https://www.governorsballmusicfestival.com/lineup/interactive-lineup/") as url:
    s = url.read()
soup = BeautifulSoup(s, "lxml")


In [ ]:
info = soup.findAll('div', {"class":"c-lineup__artist"})
#Put those names in a list, then a dataframe
govs_ball_data = pd.DataFrame([])
for i in info:
    artist = i.attrs['data-title']
    date = i.attrs["data-day-titles"]
    govs_ball_data = govs_ball_data.append(pd.DataFrame({'artist': artist, #Create the table
                                                         'date'  : date},
                                                  index=[0]),
                                     ignore_index=True)
govs_ball_data.head()

In [ ]:
govs_ball_data["day"] = ((govs_ball_data["date"].str.slice(-5, -4)).astype(int) -1).astype(str)
govs_ball_data["date"] = govs_ball_data["date"].str.slice(2, -2)
govs_ball_data

#### Add Names to a SQLite Database

In [ ]:
con = sqlite3.connect("pitchfork-data.db")

In [ ]:
govs_ball_data.to_sql("govs_ball_data", con,if_exists='replace')
con.close() #close db connection

### Load Progress Bar

In [100]:
for i in log_progress(range(1,10), every=1): #sample progress bar
    time.sleep(0.5)

## Pitchfork Crawler

The crawling happens in two distinct stages. In Stage 1, the code loops throught the reviews page on pitchfork.com to find links to all the reviews. Stage 2 goes to each link and pull various bits of information. There's lots more to pull, but this is a solid starting place. 

### Stage 1: 

In [94]:
#Stage 1
con = sqlite3.connect("pitchfork-data.db") #connect to db

for i in log_progress(range(1198,1250), every=1): 
    #Use the range function to decide how many pages you want to go through
    #In this case, I'm going through the latest 15,000 reviews (12 per page)
    page_no = str(i)
    link = ('http://pitchfork.com/reviews/albums/?page=' + page_no) #create the link
    t0 = time.time()
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    response_delay = time.time() - t0
    time.sleep(10*response_delay)  # wait 10x longer than it took them to respond
    soup = BeautifulSoup(webpage, "lxml") #create the soup
    info = soup.findAll('a', {"class":"album-link"}) #pull the album link
    for j in info:
            pd.DataFrame({'link': j.attrs['href']}, #Create the table
                         index=[0]).to_sql("link_table",
                                           con,
                                           if_exists = "append")
    clear_output() #clear ouput before rewriting progress
    print (i)

con.close() #close db connection

1249


### Stage 2:

In [ ]:
BASE_URL = 'http://www.pitchfork.com'
con = sqlite3.connect("pitchfork-data.db")
links_table = pd.read_sql_query("SELECT * from link_table", con)
links = links_table["link"]
iterator = 0

class Timeout(Exception):  # handles timeout errors (e.g., server request is taking too long)
    pass

for i in log_progress(links[7467:], every=1):
    link = BASE_URL + i
    t0 = time.time()
    req = Request(link, headers={ 'User-Agent': 'Firefox/24.0' })
    webpage = urlopen(req).read()
    response_delay = time.time() - t0
    time.sleep(8*response_delay)  # wait 10x longer than it took them to respond
    soup = BeautifulSoup(webpage, "lxml") #same as above
    artist_info = soup.findAll('ul', {"class":"artist-links artist-list"}) #Artist Name
    album_info = soup.findAll('h1', {"class":"review-title"}) #Album Name
    score_info = soup.findAll('div', {"class":"score-circle"}) #Score
    pub_info = soup.findAll('span', {"class":"pub-date"}) # Publication Date
    genre_info = soup.findAll('ul', {"class":"genre-list before"}) #Genre
    for j in artist_info:
        artist = j.text
    for k in album_info:
        album = k.text
    for l in score_info:
        score = l.text
    for m in pub_info:
        pub_date = m.text
    for n in genre_info:
        genre = n.text
    clear_output()
    print (artist, ", ", album, response_delay, iterator)
    pd.DataFrame({'artist': artist, #Create the table
                  'album'  : album,
                  'score' : score,
                  'pub_date' : pub_date,
                  'genre' : genre,
                  'link' : link},
                 index=[0]).to_sql("album_table",
                                   con,
                                   if_exists = "append")
    iterator= iterator +1

con.close() 

## Data Management

In [83]:
con = sqlite3.connect("pitchfork-data.db")
#Pull the table we just wrote back it
album_table = pd.read_sql_query("SELECT * from album_table", con)
#Drop any duplicates that may have happened
album_table = album_table.drop_duplicates()
#Delete the index
del album_table["index"]
#reupload as album_table_clean
album_table.to_sql("album_table_clean", con, if_exists = "replace")